<a href="https://colab.research.google.com/github/jvishnuvardhan/Stackoverflow_Questions/blob/master/Loading_subclassed_model_in_tf_format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tf-nightly

     |████████████████████████████████| 517.2MB 28kB/s 
     |████████████████████████████████| 2.9MB 37.7MB/s 
     |████████████████████████████████| 460kB 47.9MB/s 


In [2]:
import tensorflow as tf
tf.__version__

'2.2.0-dev20200406'

In [3]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]

In [0]:
class WideAndDeepModel(keras.models.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation, name='Dense-1')
        self.hidden2 = keras.layers.Dense(units, activation=activation, name='Dense-2')
        self.main_output = keras.layers.Dense(1, name='main_output')
        self.aux_output = keras.layers.Dense(1, name='aux_output')
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model = WideAndDeepModel(30, activation="relu")

In [9]:
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit((X_train_A, X_train_B), (y_train, y_train), epochs=1)
model.save_weights('model_weights', save_format='tf')
model.save_weights('model_weights_h5.h5', save_format='h5')

363/363 [==============================] - 0s 1ms/step - loss: nan - output_1_loss: nan - output_2_loss: nan


In [0]:
new_model = WideAndDeepModel(30, activation="relu")
new_model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))


In [8]:
# This initializes the variables used by the optimizers,
# as well as any stateful metric variables
new_model.train_on_batch((X_train_A[:1], X_train_B[:1]),((y_train[:1], y_train[:1])))

# Load the state of the old model (to load weights for all layers)
new_model.load_weights('model_weights')

In [0]:
new_model_h5 = WideAndDeepModel(30, activation="relu")
new_model_h5.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))

new_model_h5.train_on_batch((X_train_A[:1], X_train_B[:1]),((y_train[:1], y_train[:1])))
# Load the state of the old model (to load weights for all layers)
new_model_h5.load_weights('model_weights_h5.h5',by_name = True)

In [0]:
TL/DR: So if you want to load a model saved in *.tf format, then select `by_name=False`. If you want to load a model saved in *.h5 format, then select `by_name=True`. Please check the [gist here](https://colab.research.google.com/gist/jvishnuvardhan/c0c9808471249fb63298d9273b827f1e/38251-tf-nightly.ipynb).

There is small difference between loading a saved model when the model was saved in *.tf format (tensorflow format) or in *.h5 format. The following error clearly shows what we need to do when we load a tensorflow-formatted weights (it need by_name =False). 

`NotImplementedError: Weights may only be loaded based on topology into Models when loading TensorFlow-formatted weights (got by_name=True to load_weights).`

Check the description of `load_weights` on [TF website](https://www.tensorflow.org/api_docs/python/tf/keras/Model?version=nightly#load_weights). 

`Only topological loading (by_name=False) is supported when loading weights from the TensorFlow format. Note that topological loading differs slightly between TensorFlow and HDF5 formats for user-defined classes inheriting from tf.keras.Model: HDF5 loads based on a flattened list of weights, while the TensorFlow format loads based on the object-local names of attributes to which layers are assigned in the Model's constructor. `